# TensorLy: A Python Library For Tensor Learning 

TensorLy is an open-source Python library that eases the task of performing tensor operations. It provides a high-level API for dealing with deep tensorized neural networks and tensor methods. It was created in 2015 by a senior research scientist at NVIDIA Research Group named Jean Kossaifi. It was presented at the NeurIPS workshop and was later published byJean Kossaifi, Yannis Panagakis, Anima Anandkumar and Maja Pantic in a JMLR paper in February 2019.

To read about it more, please refer [this](https://analyticsindiamag.com/guide-to-tensorly-a-python-library-for-tensor-learning/) article.

# Installation of TensorLy

**NOTE:** TensorLy has been developed and tested only on Python 3 version.

Install the library using pip command:

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy sklearn statsmodels tensorflow keras --user -q

In [ ]:
!python -m pip install tensorly --user -q

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## Practical Implementation

## Tensor regression example

  Import required libraries

In [ ]:
import matplotlib.pyplot as plt
from tensorly.base import tensor_to_vec, partial_tensor_to_vec
from tensorly.datasets.synthetic import gen_image
from tensorly.random import random_tensor
from tensorly.regression.cp_regression import CPRegressor
import tensorly as tl

# Parameter of the experiment
img_ht = 25
img_width = 25
# shape of the images
patterns = ['swiss', 'circle']
# ranks to test
ranks = [1, 2, 3, 4, 5]

# Generate random samples
rng = random_tensor(1)
X = tl.tensor(rng.normal(size=(1000, img_ht, img_width), loc=0, scale=1))


# Parameters of the plot, deduced from the data
n_rows = len(patterns)
n_columns = len(ranks) + 1
# Plot the three images
fig = plt.figure()

for i, pattern in enumerate(patterns):

    # Generate the original image
    wt_img = gen_image(region=pattern, image_height=img_ht, image_width=img_width)
    wt_img = tl.tensor(wt_img)

    # Generate the labels
    y = tl.dot(partial_tensor_to_vec(X, skip_begin=1), tensor_to_vec(wt_img))

    # Plot the original weights
    ax = fig.add_subplot(n_rows, n_columns, i*n_columns + 1)
    ax.imshow(tl.to_numpy(wt_img), cmap=plt.cm.OrRd, interpolation='nearest')
    ax.set_axis_off()
    if i == 0:
        ax.set_title('Original\nweights')

    for j, rank in enumerate(ranks):

        # Create a tensor Regressor estimator
        est = CPRegressor(weight_rank=rank, tol=10e-7, n_iter_max=100, reg_W=1, verbose=0)

        # Fit the estimator to the data
        est.fit(X, y)

        ax = fig.add_subplot(n_rows, n_columns, i*n_columns + j + 2)
        ax.imshow(tl.to_numpy(est.weight_tensor_), cmap=plt.cm.OrRd, interpolation='nearest')
        ax.set_axis_off()

        if i == 0:
            ax.set_title('Learned\nrank = {}'.format(rank))

plt.suptitle("CP tensor regression")
plt.show()